In [1]:
from linked_stack import LinkedStack
from linked_queue import LinkedQueue
from tree import Tree

# 基于单链表的栈

In [2]:
class LinkedStack():
    
    class _Node():
        __slots__='_elem','_next' #slots : 槽
        
        def __init__(self,elem,next):
            self._elem=elem
            self._next=next
            
    def __init__(self):
        '''create an empty linked list'''
        self._head=None
        self._size=0
        
    def __len__(self):
        return self._size
    
    def is_empty(self):
        return len(self)==0 #self._size==0 or self._head==None
    
    def push(self,e):
        new=self._Node(e,self._head)
        self._head=new
        self._size+=1
        
    def pop(self):
        if self.is_empty():
            raise Empty('Stack is empty!')
        result=self._head._elem
        self._head=self._head._next
        self._size-=1
        return result
    
    def top(self):
        if self.is_empty():
            raise Empty('Stack is empty!')
        return self._head._elem
    
    def __repr__(self):
        data=[]
        walk=self._head
        while walk._next is not None:
            data.append(walk._elem)
            walk=walk._next
        data.append(walk._elem)
        return f'{data}'

# 基于单链表的队列

In [3]:
class LinkedQueue():
    
    class _Node():
        __slots__='_elem','_next'
        
        def __init__(self,elem,next):
            self._elem=elem
            self._next=next
            
    def __init__(self):
        self._head=None
        self._tail=None
        self._size=0
        
    def __len__(self):
        return self._size
    
    def is_empty(self):
        return len(self)==0
    
    def dequeue(self):
        if self.is_empty():
            raise Empty('Stack is empty!')
        result=self._head._elem
        self._head=self._head._next
        self._size-=1
        if self.is_empty():
            self._tail=None
        return result
    
    def first(self):
        if self.is_empty():
            raise Empty('Stack is empty!')
        return self._head._elem
    
    def enqueue(self,e):
        new=self._Node(e,None)
        if self.is_empty():
            self._head=new
        else:
            self._tail._next=new
        self._tail=new
        self._size+=1
            
    def __repr__(self):
        data=[]
        walk=self._head
        while walk._next is not None:
            data.append(walk._elem)
            walk=walk._next
        data.append(walk._elem)
        return f'{data}'

# 树的抽象数据类型

In [4]:
class Tree():
    class Position():
        def elem(self):
            raise NotImplementedError('must be implemented by subclass')
            
        def __eq__(self):
            raise NotImplementedError('must be implemented by subclass')
            
        def __ne__(self):
            raise NotImplementedError('must be implemented by subclass')
            
    def root(self):
        raise NotImplementedError('must be implemented by subclass')
        
    def parent(self,p):
        raise NotImplementedError('must be implemented by subclass')
        
    def num_children(self,p):
        raise NotImplementedError('must be implemented by subclass')
        
    def children(self,p):
        raise NotImplementedError('must be implemented by subclass')
        
    def __len__(self):
        raise NotImplementedError('must be implemented by subclass')
        
    def is_empty(self):
        return len(self)==0
    
    def is_root(self,p):
        return self.root()==p
    
    def is_leaf(self,p):
        return self.num_children(p)==0
    
    def depth(self,p):
        if self.is_root(p):
            return 0
        else:
            return 1+self.depth(self.parent(p))
        
    def _height1(self,p):
        return max(self.depth(p) for p in self.positions() if self.is_root(p))
    
    def _height2(self,p):
        if self.is_leaf(p):
            return 0
        else:
            return 1+max(self._height2(c) for c in self.children(p))
        
    def height(self,p=None):
        if p is None:
            p=self.root()
        return self._height2(p)
    
    def _subtree_preorder(self,p):
        yield p
        for c in self.children(p):
            for other in self._subtree_preorder(c):
                yield other
                
    def preorder(self):
        if not self.is_empty():
            for p in self._subtree_preorder(self.root()):
                yield p
                
    def _subtree_postorder(self,p):
        for c in self.children(p):
            for other in self._subtree_postorder(c):
                yield other
        yield p
        
    def postorder(self):
        if not self.is_empty():
            for p in self._subtree_postorder(self.root()):
                yield p
                
    def breadthfirst(self):
        if not self.is_empty():
            Q=LinkedQueue()
            Q.enqueue(self.root())
            while not Q.is_empty():
                p=Q.dequeue()
                yield p
                for c in self.children(p):
                    Q.enqueue(c)
                    
    def positions(self,mode='preorder'):
        if mode=='preorder':
            return self.preorder()
        elif mode=='postorder':
            return self.postorder()
        elif mode=='breadthfirst':
            return self.breadthfirst()
        else:
            raise ValueError('Invalid mode!')
            
    def __iter__(self,mode='preorder'):
        for p in self.positions(mode):
            yield p.elem()

# 基于树的抽象数据类型的一般树的链式结构实现

In [5]:
class LinkedGeneralTree(Tree):
    class _Node():
        __slots__='_element','_parent','_children'
        
        def __init__(self,element,parent=None,children=[]):
            self._element=element
            self._parent=parent
            self._children=children
            
    class Position(Tree.Position):
        def __init__(self,container,node):
            self._container=container
            self._node=node
            
        def elem(self):
            return self._node._element
        
        def __eq__(self,other):
            return type(self) is type(other) and self._node==other._node
        
    def _validate(self,p):
        if not isinstance(p,self.Position):
            raise TypeError('p must be proper position type!')
        if p._container is not self:
            raise ValueError('p does not belong to this container!')
        if p._node._parent is p._node:
            raise ValueError('p is not valid!')
        return p._node
    
    def _make_position(self,node):
        return self.Position(self,node) if node is not None else None
    
    def __init__(self):
        self._root=None
        self._size=0
        
    def __len__(self):
        return self._size
    
    def root(self):
        return self._make_position(self._root)
    
    def parent(self,p):
        node=self._validate(p)
        return self._make_position(node._parent)
    
    def children(self,p):
        node=self._validate(p)
        children=[]
        for x in node._children:
            children.append(self._make_position(x))
        return children
            
    def num_children(self,p):
        node=self._validate(p)
        return len(node._children)
    
    def add_root(self,e):
        if self.root() is not None:
            raise ValueError('root has already exists!')
        self._size=1
        self._root=self._Node(e)
        return self._make_position(self._root)
    
    def add_children(self,p,L):
        node=self._validate(p)
        if not node.is_leaf():
            raise ValueError('p is not a leaf!')
        self._size+=len(L)
        children=[]
        for i in range(len(L)):
            node._child=self._Node(L[i],node)
            children.append(self._make_position(node._child))
        node._children=children
        return children
    
    def attach_subtrees(self,p,subtrees):
        node=self._validate(p)
        if not self.is_leaf(p):
            raise ValueError('p is not a leaf!')
        node._children=[]
        for i in range(len(subtrees)):
            if not type(self) is type(subtrees[i]):
                raise TypeError('tree types must match!')
            self._size+=len(subtrees[i])
            if not subtrees[i].is_empty():
                subtrees[i]._root._parent=node
                node._children.append(subtrees[i]._root)
                subtrees[i]._root=None
                subtrees[i]._size=0
            
    def replace(self,p,e):
        node=self._validate(p)
        old_value=node._element
        node._element=e
        return old_value
    
    def delete(self,p):
        node=self._validate(p)
        if self.num_children(p)>1:
            raise ValueError('Position has over one children!')
        children=list(self.children(node))
        for child in children:
            if child is not None:
                child._parent=node._parent
            if node is self._root:
                self._root=child
            else:
                parent=node._parent
                parent._children1=[child if x==node else x for x in parent._children]
                parent._children=parent._children1
        self._size-=1
        node._parent=node
        return node._element
    
## preorder,postorder,levelorder has already been written in superclass -- Tree
    
    def _subtree_preorder(self,p):
        yield p
        for c in self.children(p):
            for other in self._subtree_preorder(c):
                yield other
                
    def preorder(self):
        if not self.is_empty():
            for p in self._subtree_preorder(self.root()):
                yield p
                
    def _subtree_postorder(self,p):
        for c in self.children(p):
            for other in self._subtree_postorder(c):
                yield other
        yield p
        
    def postorder(self):
        if not self.is_empty():
            for p in self._subtree_postorder(self.root()):
                yield p
                
    def levelorder(self):
        if not self.is_empty():
            Q=LinkedQueue()
            Q.enqueue(self.root())
            while not Q.is_empty():
                p=Q.dequeue()
                yield p
                for c in self.children(p):
                    Q.enqueue(c)
    
    def reverse_levelorder(self):
        if not self.is_empty():
            Q=LinkedQueue()
            S=LinkedStack()
            Q.enqueue(self.root())
            while not Q.is_empty():
                s=Q.dequeue()
                S.push(s)
                for c in self.children(s):
                    Q.enqueue(c)
            while not S.is_empty():
                a=S.pop()
                yield a
                    
    def k_distance(self,k):
        if self.is_empty():
            raise ValueError('there is no root!')
        if k>self.height() or k<0:
            raise ValueError('k is not suitable for the height of the root!')
        else:
            Q=LinkedQueue()
            Q.enqueue(self.root())
            n=0
            while n<k:
                for i in range(len(Q)):
                    q=Q.dequeue()
                    for c in self.children(q):
                        Q.enqueue(c)
                n+=1
            while not Q.is_empty():
                x=Q.dequeue()
                yield x

# 测试阶段

一般链式结构树的构造

In [6]:
t1=LinkedGeneralTree()
t1.add_root('G')
t2=LinkedGeneralTree()
t2.add_root('H')
t3=LinkedGeneralTree()
t3.add_root('E')
t3.attach_subtrees(t3.root(),[t1,t2])

In [7]:
t4=LinkedGeneralTree()
t4.add_root('D')
t5=LinkedGeneralTree()
t5.add_root('F')
t6=LinkedGeneralTree()
t6.add_root('B')
t6.attach_subtrees(t6.root(),[t4,t3,t5])

In [8]:
t7=LinkedGeneralTree()
t7.add_root('C')
t8=LinkedGeneralTree()
t8.add_root('A')
t8.attach_subtrees(t8.root(),[t6,t7])

In [9]:
print(type(t8.root()))

<class '__main__.LinkedGeneralTree.Position'>


前序遍历的实现

In [10]:
L1=t8.preorder()

In [11]:
# 前序遍历的实现
for k in range(len(t8)):
    print(next(L1).elem())

A
B
D
E
G
H
F
C


后续遍历的实现

In [12]:
L2=t8.postorder()

In [13]:
#后序遍历的实现
for k in range(len(t8)):
    print(next(L2).elem())

D
G
H
E
F
B
C
A


层序遍历的实现

In [14]:
L3=t8.levelorder()

In [15]:
#层序遍历的实现
for k in range(len(t8)):
    print(next(L3).elem())

A
B
C
D
E
F
G
H


逆层序遍历的实现

In [16]:
L4=t8.reverse_levelorder()

In [17]:
#逆层序遍历的实现
for k in range(len(t8)):
    print(next(L4).elem())

H
G
F
E
D
C
B
A


到根节点的距离为k的节点位置的迭代

In [18]:
# k_distance的检验
y0=t8.k_distance(0)
for x in y0:
    print(x.elem())

A


In [19]:
y1=t8.k_distance(1)
for x in y1:
    print(x.elem())

B
C


In [20]:
y2=t8.k_distance(2)
for x in y2:
    print(x.elem())

D
E
F


In [21]:
y3=t8.k_distance(3)
for x in y3:
    print(x.elem())

G
H


In [24]:
t8.num_children(t8.root())

2

In [27]:
t8.num_children(t8.children(t8.root())[0])

3